In [1]:
import pandas as pd
import numpy as np
import cv2
import dlib
import math
from time import time

In [2]:
df=pd.read_csv("../facial_expressions/data/legend.csv").drop("user.id",axis=1)

In [3]:
IMG_PATH="../facial_expressions/images/"

In [4]:
emotions=["neutral", "anger", "disgust", "fear", "happy", "sadness"]

In [5]:
df["emotion"]=df["emotion"].map(lambda x: x.lower())

In [6]:
df=df.replace(to_replace ="contempt", value ="disgust")
df=df.replace(to_replace ="surprise", value ="happy")
df=df.replace(to_replace ="happiness", value ="happy")
df=df.replace(to_replace ="sad", value ="sadness")

In [7]:
inds=[]
for em in df["emotion"]:
    inds.append(emotions.index(em))
df["index"]=inds

In [8]:
df

,image,emotion,index
0,facial-expressions_2868588k.jpg,anger,1
1,facial-expressions_2868585k.jpg,happy,4
2,facial-expressions_2868584k.jpg,disgust,2
3,facial-expressions_2868582k.jpg,fear,3
4,Aaron_Eckhart_0001.jpg,neutral,0
...,...,...,...
14185,Tim_Pawlenty_0001.jpg,neutral,0
14186,Tim_Robbins_0001.jpg,neutral,0
14187,Tim_Robbins_0002.jpg,neutral,0
14188,Tim_Robbins_0003.jpg,neutral,0


In [9]:
face_cascade = cv2.CascadeClassifier('../face_recognition/haarcascade_frontalface_default.xml')
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [10]:
face_dim=64

In [11]:
def ret_keypoints(imname):
    try:
        image = cv2.imread(IMG_PATH+imname)
    except:
        return None
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (face_dim, face_dim)) 
#     faces = face_cascade.detectMultiScale(gray, 1.3, 5)
#     boxes = [(y,x+w,y+h,x) for (x,y,w,h) in faces]
    shape=predictor(gray,dlib.rectangle(0,0,face_dim,face_dim))
    xlist=[]
    ylist=[]
    for i in range(68):
        xlist.append(float(shape.part(i).x))
        ylist.append(float(shape.part(i).y))
    xmean = np.mean(xlist)
    ymean = np.mean(ylist)
    xcentral = [(x-xmean) for x in xlist]
    ycentral = [(y-ymean) for y in ylist]
    res=[]
    for (x,y) in zip(xcentral,ycentral):
        res.append(x)
        res.append(y)
    return (np.asarray(res)/face_dim + 1)/2

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

In [15]:
model=Sequential()
model.add(Dense(1024,activation='relu', input_shape=(68*2,)))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(len(emotions),activation='softmax'))

InternalError: cudaGetDevice() failed. Status: CUDA driver version is insufficient for CUDA runtime version

In [ ]:
model.summary()

In [14]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [14]:
df.iloc[0:10]

,image,emotion,index
0,facial-expressions_2868588k.jpg,anger,1
1,facial-expressions_2868585k.jpg,happy,4
2,facial-expressions_2868584k.jpg,disgust,2
3,facial-expressions_2868582k.jpg,fear,3
4,Aaron_Eckhart_0001.jpg,neutral,0
5,Aaron_Guiel_0001.jpg,happy,4
6,Aaron_Patterson_0001.jpg,neutral,0
7,Aaron_Peirsol_0001.jpg,happy,4
8,Aaron_Peirsol_0002.jpg,happy,4
9,Aaron_Peirsol_0003.jpg,happy,4


In [17]:
def prep_keypoints():
    X_inp=[]
    y_inp=[]
    for idx in range(len(df)):
        print("\r",idx+1,end=" ")
        row=df.iloc[idx]
        landmarks=ret_keypoints(row["image"])
        if landmarks is not None:
            X_inp.append(landmarks)
            yy=np.zeros(len(emotions))
            yy[row["index"]]=1
            y_inp.append(yy)
    return np.asarray(X_inp),np.asarray(y_inp)

In [19]:
XDB,YDB=prep_keypoints()

 14189 

In [22]:
BATCH_SIZE=64
GCT=0
LN=len(XDB)

In [23]:
def get_batch():
    global GCT
    X_inp=XDB[GCT:GCT+BATCH_SIZE]
    y_inp=YDB[GCT:GCT+BATCH_SIZE]
    GCT=(GCT+1)%LN
    return X_inp,y_inp

In [176]:
def trainer():
    compe=len(XDB)//BATCH_SIZE + 1
    for i in range(compe):
        print("\r"+str((i+1)/compe*100)[:6],end=" %    ")
        X_inp,y_inp=get_batch()
        model.train_on_batch(X_inp,y_inp)

In [112]:
EPOCHS=10
for ep in range(EPOCHS):
    print("EPOCH:",ep+1,"/",EPOCHS)
    stt=time()
    trainer()
    print("\tTime taken:",str(time()-stt)[:5],"s")

EPOCH: 1 / 10
100.0 %     	Time taken: 64.76 s
EPOCH: 2 / 10
100.0 %     	Time taken: 61.47 s
EPOCH: 3 / 10
100.0 %     	Time taken: 59.10 s
EPOCH: 4 / 10
100.0 %     	Time taken: 60.76 s
EPOCH: 5 / 10
44.594 %    

KeyboardInterrupt: 

In [173]:
a,b=get_batch(df)

In [174]:
(model.predict(a).argmax(axis=1)==b.argmax(axis=1)).sum()/64

0.6875

In [145]:
model.predict(a).argmax(axis=1)

array([0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0,
       0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 4, 0, 0,
       0, 0, 0, 0, 0, 0, 4, 0, 4, 4, 0, 0, 4, 4, 0, 4, 4, 4, 4, 0])

In [146]:
b.argmax(axis=1)

array([0, 0, 0, 0, 0, 0, 4, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0,
       0, 4, 0, 4, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 4, 0, 0,
       0, 0, 0, 2, 0, 0, 4, 0, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 0])